## CSE 8673: Machine Learning
## Homework 2
## Zach Douglas

Read training and label data, convert data from string to array of integers.

In [7]:
with open('mnist_data.txt') as f:
    images = f.readlines()

with open('mnist_labels.txt') as f:
    labels = f.readlines()

In [8]:
for digit in images:
    images[images.index(digit)] = [int(pixels) for pixels in digit.split()]
    
for string in labels:
    labels[labels.index(string)] = int(labels[labels.index(string)])

Import modules for the following functions.

In [9]:
from random import randint
from random import choice
from statistics import mode

Function to return the Euclidean distance between two vectors of magnitude 784.

In [10]:
def distance(sample, centroid):
    temp = []
    
    for i in range(0, 784):
        difference = centroid[i] - sample[i]
        sq_diff = difference**2
        temp.append(sq_diff)
        
    total = sum(temp)
    
    return total**0.5

Function to return the mean of an array of vectors of magnitude 784.

In [11]:
def mean(cluster):
    cluster_mean = []
    cluster_sum = [sum(pixels) for pixels in zip(*cluster)]
    cluster_length = 784

    for i in range(0, cluster_length):
        cluster_mean.append(cluster_sum[i]/len(cluster))

    return cluster_mean

K-means Algorithm

In [6]:
def k_means(data, k):
    centroid_history = [[]]
    centroid_list = []

    for i in range (0, k):
        random_sample = randint(0, len(data))
        centroid_list.append(data[random_sample])
        centroid_history[0].append(data[random_sample])
  
    while (len(centroid_history) == 1 or centroid_history[-1]!=centroid_history[-2]):
        centroid_assignment = []
        
        for i in range(0, len(data)):
            dist_list = []

            for j in range(0, len(centroid_list)):
                dist = distance(centroid_list[j], data[i])
                dist_list.append(dist)

            closest_dist = min(dist_list)
            index_of_closest_centroid = dist_list.index(closest_dist)
            closest_centroid = centroid_list[index_of_closest_centroid]

            centroid_assignment.append(centroid_list.index(closest_centroid))
      
        for i in range(0, len(centroid_list)):
            cluster_list = []
        
            for j in range(0, len(centroid_assignment)):
                if centroid_assignment[j] == i:
                    cluster_list.append(data[j])
    
            if len(cluster_list) > 0:
                cluster_mean = mean(cluster_list)
                centroid_list[i] = cluster_mean
    
        centroid_history.append(centroid_list.copy())

    return centroid_assignment, centroid_history

Function to return a nested array of length 10 (one item for each cluster). Each item in the array contains a list of indices for the respective cluster label.

In [12]:
def get_indices_10(assignments):
    indices = [[],[],[],[],[],[],[],[],[],[]]

    for i in range(0, len(assignments)):
        if assignments[i] == 0:
            indices[0].append(i)
        elif assignments[i] == 1:
            indices[1].append(i)
        elif assignments[i] == 2:
            indices[2].append(i)
        elif assignments[i] == 3:
            indices[3].append(i)
        elif assignments[i] == 4:
            indices[4].append(i)
        elif assignments[i] == 5:
            indices[5].append(i)
        elif assignments[i] == 6:
            indices[6].append(i)
        elif assignments[i] == 7:
            indices[7].append(i)
        elif assignments[i] == 8:
            indices[8].append(i)
        elif assignments[i] == 9:
            indices[9].append(i)
            
    return indices

Function to return the number of samples which do not have the label of the majority (number of minority samples).

In [13]:
def num_minorities(k_indices, original_labels):
    cluster_labels = []
    sum = 0
    for index in k_indices:
        cluster_labels.append(original_labels[index])

    cluster_mode = mode(cluster_labels)

    not_mode = 0
    for label in cluster_labels:
        if label != cluster_mode:
            not_mode += 1

    return not_mode

Function to return the sum of minority samples across 10 clusters.

In [14]:
def minority_sum(all_indices, original_labels):
    total = 0
    
    for i in range(0, 10):
        total += num_minorities(all_indices[i], original_labels)

    return total

Run K-mean algorithm for K=10 for 10 iterations, calculate average minority samples and average iterations required for algorithm to converge.

In [10]:
all_sums = []
num_convergences = []

In [11]:
for i in range(0, 10):
    assignments, hist = k_means(images, 10)
    indices = get_indices_10(assignments)
    all_sums.append(minority_sum(indices, labels))
    num_convergences.append(len(hist))

In [12]:
print(all_sums, num_convergences)

[4075, 4062, 3725, 4685, 3920, 4163, 3950, 3952, 3965, 4031] [30, 71, 68, 98, 35, 33, 70, 102, 55, 60]


In [13]:
mean_wrong_cluster = sum(all_sums)/100
mean_num_convergences = sum(num_convergences)/10
print(mean_wrong_cluster, mean_num_convergences)

405.28 62.2


Alter previous function to return indices when K=5.

In [14]:
def get_indices_5(assignments):
    indices = [[],[],[],[],[]]

    for i in range(0, len(assignments)):
        if assignments[i] == 0:
            indices[0].append(i)
        elif assignments[i] == 1:
            indices[1].append(i)
        elif assignments[i] == 2:
            indices[2].append(i)
        elif assignments[i] == 3:
            indices[3].append(i)
        elif assignments[i] == 4:
            indices[4].append(i)
      
    return indices

Function to return a dictionary, keys are the hand-written digits (0-9), values are how many of that digit are in the cluster.

In [18]:
def digit_count(k_indices, original_labels):
    cluster_labels = []
    digit_count = {}
    sum = 0
    
    for index in k_indices:
        cluster_labels.append(original_labels[index])

    for i in range(0, 10):
        digit_count[i] = cluster_labels.count(i)

    return digit_count

Running K-means algorithm again, this time with K=5.

In [16]:
assignments, hist = k_means(images, 5)

In [17]:
indices = get_indices_5(assignments)

Showing the counts of each digit for each cluster.

In [18]:
digit_count(indices[0], labels)

{0: 26, 1: 2, 2: 30, 3: 54, 4: 338, 5: 216, 6: 1, 7: 598, 8: 336, 9: 386}

In [19]:
digit_count(indices[1], labels)

{0: 112, 1: 3, 2: 625, 3: 19, 4: 45, 5: 24, 6: 819, 7: 7, 8: 30, 9: 15}

In [20]:
digit_count(indices[2], labels)

{0: 3, 1: 0, 2: 21, 3: 52, 4: 557, 5: 77, 6: 9, 7: 333, 8: 88, 9: 564}

In [21]:
digit_count(indices[3], labels)

{0: 829, 1: 2, 2: 88, 3: 655, 4: 0, 5: 436, 6: 34, 7: 0, 8: 256, 9: 15}

In [22]:
digit_count(indices[4], labels)

{0: 10, 1: 1128, 2: 268, 3: 230, 4: 42, 5: 139, 6: 95, 7: 90, 8: 264, 9: 29}

Reinitialize centroids for K-means Algorithm

In [5]:
from tqdm import tqdm

def k_means(data, k, original_labels):
    centroid_history = [[]]
    centroid_list = []

    label_indices = get_indices_10(original_labels)
  
    for i in range(0, k):
        random_index = choice(label_indices[i])
        centroid_list.append(data[random_index])
        centroid_history[0].append(data[random_index])
  
    while (len(centroid_history) == 1 or centroid_history[-1]!=centroid_history[-2]):
        centroid_assignment = []
        
        for i in tqdm(range(0, len(data))):
            dist_list = []

            for j in range(0, len(centroid_list)):
                dist = distance(centroid_list[j], data[i])
                dist_list.append(dist)

            closest_dist = min(dist_list)
            index_of_closest_centroid = dist_list.index(closest_dist)
            closest_centroid = centroid_list[index_of_closest_centroid]

            centroid_assignment.append(centroid_list.index(closest_centroid))
      
        for i in tqdm(range(0, len(centroid_list))):
            cluster_list = []

            for j in range(0, len(centroid_assignment)):
                if centroid_assignment[j] == i:
                    cluster_list.append(data[j])
    
            if len(cluster_list) > 0:
                cluster_mean = mean(cluster_list)
                centroid_list[i] = cluster_mean
    
        centroid_history.append(centroid_list.copy())
    
    return centroid_assignment, centroid_history

Re-run K-mean algorithm for K=10 for 10 iterations (this time with centroids initialized for digits 0-9), calculate average minority samples and average iterations required for algorithm to converge.

In [24]:
all_sums = []
num_convergences = []

In [25]:
for i in range(0, 10):
    assignments, hist = k_means(images, 10, labels)
    indices = get_indices_10(assignments)
    all_sums.append(minority_sum(indices, labels))
    num_convergences.append(len(hist))

In [26]:
print(all_sums, num_convergences)

[4015, 4599, 3645, 3899, 3574, 4074, 4089, 3946, 3958, 3655] [40, 28, 32, 49, 35, 46, 33, 55, 65, 51]


In [27]:
mean_wrong_cluster = sum(all_sums)/100
mean_num_convergences = sum(num_convergences)/10
print(mean_wrong_cluster, mean_num_convergences)

394.54 43.4


In [15]:
assignments, hist = k_means(images, 10, labels)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 51.38it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 56.87it/s]


In [16]:
indices = get_indices_10(assignments)

In [19]:
digit_count(indices[0], labels)

{0: 719, 1: 0, 2: 13, 3: 0, 4: 1, 5: 4, 6: 20, 7: 2, 8: 7, 9: 6}

In [20]:
digit_count(indices[1], labels)

{0: 3, 1: 1102, 2: 133, 3: 52, 4: 27, 5: 84, 6: 40, 7: 67, 8: 72, 9: 23}

In [21]:
digit_count(indices[2], labels)

{0: 1, 1: 23, 2: 725, 3: 32, 4: 0, 5: 2, 6: 5, 7: 14, 8: 11, 9: 2}

In [22]:
digit_count(indices[3], labels)

{0: 53, 1: 3, 2: 41, 3: 690, 4: 0, 5: 280, 6: 4, 7: 0, 8: 107, 9: 7}

In [23]:
digit_count(indices[4], labels)

{0: 3, 1: 0, 2: 24, 3: 14, 4: 411, 5: 37, 6: 12, 7: 126, 8: 15, 9: 333}

In [24]:
digit_count(indices[5], labels)

{0: 184, 1: 0, 2: 24, 3: 82, 4: 15, 5: 246, 6: 224, 7: 0, 8: 59, 9: 7}

In [25]:
digit_count(indices[6], labels)

{0: 13, 1: 2, 2: 15, 3: 1, 4: 17, 5: 7, 6: 649, 7: 0, 8: 5, 9: 3}

In [26]:
digit_count(indices[7], labels)

{0: 1, 1: 0, 2: 8, 3: 9, 4: 237, 5: 30, 6: 2, 7: 456, 8: 12, 9: 420}

In [27]:
digit_count(indices[8], labels)

{0: 3, 1: 4, 2: 38, 3: 121, 4: 1, 5: 139, 6: 2, 7: 1, 8: 622, 9: 11}

In [28]:
digit_count(indices[9], labels)

{0: 0, 1: 1, 2: 11, 3: 9, 4: 273, 5: 63, 6: 0, 7: 362, 8: 64, 9: 197}